In [1]:
%pip install numpy
%pip install pandas
%pip install sklearn
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from sklearn.cluster import DBSCAN

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv("water_potability.csv")
desc = df.describe()
desc

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [3]:
imputer = KNNImputer(n_neighbors=3)
df_filled = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [7]:
df_potable = df_filled.query('Potability == 1').drop(columns=['Potability'])
df_unpotable = df_filled.query('Potability == 0').drop(columns=['Potability'])

df_potable = df_potable.reset_index().drop(columns=['index'])
df_unpotable = df_unpotable.reset_index().drop(columns=['index'])

display(df_potable.describe())
display(df_unpotable.describe())

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000
mean,7.067622,195.800744,22383.991018,7.169338,332.638612,425.383800,14.160893,66.565356,3.968328
std,1.391363,35.547041,9101.010208,1.702988,43.444485,82.048446,3.263907,16.080678,0.780842
min,0.227499,47.432000,728.750830,0.352000,129.000000,201.619737,2.200000,8.175876,1.492207
25%,6.233140,174.330531,15668.985035,6.094134,306.329450,360.939023,12.033897,56.717065,3.430909
50%,7.041030,196.632907,21199.386614,7.215163,333.142134,420.712729,14.162809,66.697224,3.958576
75%,7.887966,218.003420,27973.236446,8.199261,359.655180,484.155911,16.356245,77.204861,4.509569
max,13.175402,323.124000,56488.672413,13.127000,481.030642,695.369528,23.604298,124.000000,6.494249


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000,1998.000000
mean,7.085311,196.733292,21777.490788,7.092175,334.092525,426.730454,14.364335,66.304939,3.965800
std,1.589761,31.057540,8543.068788,1.501045,34.053413,80.047317,3.334554,15.769498,0.780282
min,0.000000,98.452931,320.942611,1.683993,203.444521,181.483754,4.371899,0.738000,1.450000
25%,6.095666,177.823265,15663.057382,6.155640,313.004398,368.498530,12.101057,55.995254,3.444062
50%,7.059692,197.123423,20809.618280,7.090334,333.389426,422.229331,14.293508,66.481145,3.948076
75%,8.043015,216.120687,27006.249009,8.066462,354.052560,480.677198,16.649485,77.000519,4.496106
max,14.000000,304.235912,61227.196008,12.653362,460.107069,753.342620,28.300000,120.030077,6.739000


In [5]:
def find_outliers(data_frame):
    clf = LocalOutlierFactor(n_neighbors=3)
    outliers = clf.fit_predict(data_frame)
    outliers_indexes = np.where(outliers == - 1)[0]
    rest_indexes = np.where(outliers == 1)[0]
    outliers = data_frame.loc[outliers_indexes]
    df_without_outliers = data_frame.loc[rest_indexes]
    df_without_outliers = df_without_outliers.reset_index().drop(columns=['index'])
    return outliers, df_without_outliers

In [16]:
outliers_potable, df_pot_clean = find_outliers(df_potable)
outliers_unpotable, df_unpot_clean = find_outliers(df_unpotable)

In [17]:
display(df_pot_clean.describe())
display(df_unpot_clean.describe())


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000
mean,7.053519,196.111842,22442.044846,7.159911,332.287693,425.553339,14.187774,66.557729,3.970242
std,1.376664,34.844169,9025.039294,1.705690,43.098507,80.232716,3.254620,16.124780,0.782532
min,0.227499,73.492234,728.750830,0.352000,129.000000,217.358330,2.200000,8.175876,1.492207
25%,6.215342,174.519217,15820.386485,6.086919,306.333536,362.595921,12.043794,56.796685,3.433837
50%,7.039094,196.791251,21226.665548,7.207846,333.061939,421.099917,14.182396,66.700476,3.959294
75%,7.883639,217.807732,27980.426315,8.185234,358.837925,482.296528,16.386262,77.291925,4.510990
max,11.491011,323.124000,56488.672413,13.127000,481.030642,695.369528,23.604298,124.000000,6.494249


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000
mean,7.068952,197.156957,21758.245459,7.097606,333.889164,426.299014,14.367964,66.369303,3.960486
std,1.587592,30.525192,8383.777978,1.493119,33.168892,77.231055,3.340583,15.737804,0.778338
min,0.000000,98.452931,3413.081633,1.683993,203.444521,210.319182,4.371899,0.738000,1.450000
25%,6.086873,178.544525,15643.924307,6.167605,313.105285,369.241099,12.161396,56.006535,3.440020
50%,7.040245,197.517093,20855.272321,7.086469,333.148318,421.963539,14.298579,66.588849,3.941074
75%,8.035278,216.096228,27000.721100,8.068454,353.374790,479.164849,16.647555,77.029129,4.482006
max,14.000000,304.235912,49456.587108,12.653362,458.441072,672.556999,28.300000,120.030077,6.739000


In [34]:
display(df_pot_clean.describe())
display(outliers_potable)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000,1231.000000
mean,7.053519,196.111842,22442.044846,7.159911,332.287693,425.553339,14.187774,66.557729,3.970242
std,1.376664,34.844169,9025.039294,1.705690,43.098507,80.232716,3.254620,16.124780,0.782532
min,0.227499,73.492234,728.750830,0.352000,129.000000,217.358330,2.200000,8.175876,1.492207
25%,6.215342,174.519217,15820.386485,6.086919,306.333536,362.595921,12.043794,56.796685,3.433837
50%,7.039094,196.791251,21226.665548,7.207846,333.061939,421.099917,14.182396,66.700476,3.959294
75%,7.883639,217.807732,27980.426315,8.185234,358.837925,482.296528,16.386262,77.291925,4.510990
max,11.491011,323.124000,56488.672413,13.127000,481.030642,695.369528,23.604298,124.000000,6.494249


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
13,13.175402,47.432000,19237.949676,8.907020,375.147315,500.245952,12.083896,62.532615,4.106924
18,6.869639,251.293448,21728.821295,8.803175,279.776857,539.466877,12.994140,56.409708,4.726702
28,10.761898,81.710895,25999.953669,8.477394,318.427241,392.704082,12.716350,52.246972,4.661799
50,7.780825,225.287704,22511.692615,7.057578,377.128970,291.353701,11.590025,72.508523,4.081701
51,10.049674,132.832837,11557.032038,8.550573,323.508072,442.768050,13.376149,92.176170,5.133478
90,6.848332,159.188050,12837.506063,5.164448,373.991711,558.815567,12.254081,69.772210,4.396605
113,4.812434,250.183954,11465.575643,4.362641,398.161904,607.026542,18.210032,62.511467,3.020996
136,4.212970,193.503415,42268.334296,9.005943,258.392683,384.684875,9.085279,75.439633,3.601735
169,7.098831,186.595255,29496.910157,8.117748,378.430501,616.175243,14.962780,65.444563,2.475284
283,11.898078,228.132673,8296.550611,9.899637,346.649750,500.169210,15.486862,63.529460,3.024968


In [33]:
display(df_unpot_clean.describe())
display(outliers_unpotable)

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000,1923.000000
mean,7.068952,197.156957,21758.245459,7.097606,333.889164,426.299014,14.367964,66.369303,3.960486
std,1.587592,30.525192,8383.777978,1.493119,33.168892,77.231055,3.340583,15.737804,0.778338
min,0.000000,98.452931,3413.081633,1.683993,203.444521,210.319182,4.371899,0.738000,1.450000
25%,6.086873,178.544525,15643.924307,6.167605,313.105285,369.241099,12.161396,56.006535,3.440020
50%,7.040245,197.517093,20855.272321,7.086469,333.148318,421.963539,14.298579,66.588849,3.941074
75%,8.035278,216.096228,27000.721100,8.068454,353.374790,479.164849,16.647555,77.029129,4.482006
max,14.000000,304.235912,49456.587108,12.653362,458.441072,672.556999,28.300000,120.030077,6.739000


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
1,3.716080,129.422921,18630.057858,6.635246,351.285226,592.885359,15.180013,56.329076,4.500656
8,7.607123,118.988579,14285.583854,7.804174,268.646941,389.375566,12.706049,53.928846,3.595017
84,6.395359,133.834229,23827.275015,5.127747,266.908428,573.400295,9.833875,95.026609,4.191218
108,6.098157,189.995846,27776.901052,6.443098,347.930400,245.859632,14.636882,73.424632,3.977362
126,7.216172,252.670469,29368.674184,9.906147,433.952212,465.216621,13.221034,84.198607,5.052737
...,...,...,...,...,...,...,...,...,...
1898,11.496859,186.125296,12253.303389,8.066980,325.409341,260.531758,9.674083,68.073928,5.005193
1902,10.337671,163.276225,21820.295875,7.899521,356.179637,561.206238,14.116324,56.193061,5.063109
1951,6.129386,216.237975,20886.786563,7.326370,303.498672,269.817539,15.252780,104.682553,3.860348
1962,10.817831,141.488391,23989.990151,7.962066,389.392897,361.632055,14.908725,57.164556,3.590841


In [19]:
def clusterize(data_frame, eps):
    normalized_df=(data_frame-data_frame.min())/(data_frame.max()-data_frame.min())
    clustering = DBSCAN(eps=eps).fit(normalized_df)
    for label in range(clustering.n_features_in_):
        tmp_indexes = np.where(clustering.labels_ == label)[0]
        tmp = data_frame.loc[tmp_indexes].reset_index().drop(columns=['index'])
        if(len(tmp)):
            display(f'Cluster {label}:')
            display(f'Number of elements: {len(tmp)}')
            # print(f'Labels: {len(df_without_outliers)}')
            # print(tmp[['ph', 'Hardness', 'Potability']].describe())
            display(tmp.describe())
            # print(tmp[['ph', 'Hardness', 'Potability']])
    

In [32]:
clusterize(df_pot_clean, 0.205)

'Cluster 0:'

'Number of elements: 364'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000,364.000000
mean,7.064195,196.548159,21215.424259,7.142843,332.034598,412.236326,14.684365,66.954306,3.930120
std,0.779684,20.147471,6050.314061,1.035641,27.649366,64.084595,2.558494,12.172994,0.619287
min,4.857803,135.259272,8406.723244,4.398831,227.348460,277.412711,8.630091,32.920720,2.188702
25%,6.576727,182.881394,16797.051941,6.391832,314.629018,366.696749,12.654428,58.543810,3.493840
50%,7.063201,197.335425,20416.004579,7.162538,330.624981,409.429761,14.670400,67.175221,3.944100
75%,7.609692,211.087799,25390.526855,7.837221,351.283157,461.009716,16.596317,75.568437,4.366290
max,8.832641,249.163567,41408.114822,9.817336,409.286210,571.822068,20.638076,96.795403,5.661104


'Cluster 1:'

'Number of elements: 4'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,7.331931,224.355036,20456.046104,6.632066,346.944111,514.483961,14.510770,83.910057,5.177566
std,0.472759,6.203632,2081.304829,0.823222,23.362016,14.063251,1.110344,7.914404,0.233315
min,6.865569,216.473169,17679.351101,5.676387,324.586739,496.603425,13.787264,73.192470,4.916218
25%,6.959232,221.802147,19614.632163,6.082295,331.766935,506.686029,13.868539,80.658367,5.074988
50%,7.327623,224.750960,20779.522254,6.733232,342.660833,517.063301,14.050426,85.493024,5.156176
75%,7.700322,227.303848,21620.936195,7.283004,357.838008,524.861234,14.692657,88.744714,5.258754
max,7.806909,231.445054,22585.788809,7.385414,377.868040,527.205818,16.154964,91.461709,5.481691


'Cluster 2:'

'Number of elements: 5'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,7.376799,166.630527,37593.055129,5.470053,297.047963,496.191080,15.599436,71.503570,4.326214
std,0.768101,6.755417,2560.681519,0.387393,7.800578,20.281653,1.181436,6.338198,0.295662
min,6.642756,161.124205,34510.752995,4.984432,284.624270,474.057490,14.299242,62.756891,3.877349
25%,7.169251,162.478226,36505.524258,5.381994,295.834151,477.994992,14.395519,68.271204,4.275645
50%,7.186931,163.167250,36525.550884,5.408354,298.552480,499.275401,15.965652,71.706891,4.299209
75%,7.200280,168.896422,39624.087179,5.512927,300.695130,507.027006,16.559229,75.914398,4.533174
max,8.684779,177.486533,40799.360328,6.062561,305.533784,522.600512,16.777540,78.868465,4.645692


In [29]:
clusterize(df_unpot_clean, 0.22)

'Cluster 0:'

'Number of elements: 1012'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000,1012.000000
mean,7.085053,197.174536,21114.935337,7.063913,333.503183,423.636442,14.442333,66.058514,3.961157
std,1.238207,22.291576,6526.845943,1.163126,24.652281,65.768398,2.800883,13.287334,0.652164
min,3.261670,120.327615,5576.628297,2.750837,259.122131,266.357744,5.051695,17.915723,1.680554
25%,6.253102,182.122279,16469.595117,6.293838,316.753040,374.508360,12.469317,58.092664,3.527024
50%,7.054475,196.867829,20518.709473,7.069786,333.025546,419.327009,14.352468,66.141672,3.947064
75%,7.903980,212.309380,25457.281431,7.844186,349.674512,472.262776,16.413958,75.523137,4.409459
max,10.695532,261.566173,44652.363872,11.208688,416.530533,617.540906,22.846529,101.248253,5.989543


'Cluster 1:'

'Number of elements: 8'

...

'Number of elements: 5'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,6.790774,193.032813,19912.032760,7.763835,343.327946,456.392711,4.805907,69.536575,3.563802
std,1.198270,12.013165,2188.561543,0.549653,9.401596,31.417283,0.400637,5.383492,0.280837
min,5.122648,175.873676,17279.513749,7.027441,331.214867,428.660965,4.371899,61.911364,3.293553
25%,6.001382,186.884422,18752.269677,7.487278,336.466042,444.891044,4.466772,65.954766,3.442121
50%,7.215791,195.018888,19214.186357,7.710120,344.589628,446.712219,4.861631,72.305219,3.504525
75%,7.593177,201.068737,21759.363929,8.202453,351.386080,451.211449,4.966862,72.837540,3.542093
max,8.020869,206.318343,22554.830087,8.391883,352.983112,510.487880,5.362371,74.673986,4.036720


'Cluster 4:'

'Number of elements: 6'

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,7.223056,192.626124,30533.349603,5.509723,316.150208,468.095367,15.434972,43.042914,4.671206
std,0.414157,12.129699,2982.233847,0.543370,18.154448,32.179261,0.620019,9.221875,0.284711
min,6.490923,173.272818,25396.011782,4.737826,297.512799,436.396995,14.567819,31.555010,4.308570
25%,7.108787,188.424256,30289.783899,5.120574,303.391354,446.997623,15.035719,37.309295,4.452305
50%,7.317610,192.883258,30554.893149,5.657466,310.452178,462.347535,15.444937,43.251255,4.718112
75%,7.501279,198.942070,31604.479299,5.917254,328.857644,475.372622,15.964281,46.099791,4.831959
max,7.610509,208.817812,34538.544038,6.060119,342.453546,525.884833,16.115432,57.822023,5.048834
